In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm.auto import tqdm
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys
from collections import Counter,deque
import matplotlib.pyplot as plt
import json
import functools, itertools

In [2]:
import tensorflow as tf
import tensorflow.feature_column as fc
import numpy as np
import numpy as np
import pandas as pd
from pyhocon import ConfigFactory

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # 禁用GPU

In [4]:
DATA_PATH="/tmp-data/zhoutongzt/DD/Data"
NFS_DATA_PATH="/nfs/map-tmp-vol0/zhoutong/Data"

# 处理TFRecord

## 检查schema

In [32]:
# trd_dir="/tmp-data/zhoutongzt/DD/Data/BR_useReqTime_addCross_test"
trd_dir = NFS_DATA_PATH+"/BR_useReqTime_expTryMMOE_train"
trd_fp = [os.path.join(trd_dir,i) for i in os.listdir(trd_dir)]
trd=tf.data.TFRecordDataset(trd_fp,compression_type='GZIP')
iterator=trd.make_one_shot_iterator()
one=iterator.get_next()
res = []
with tf.Session() as sess:
    one_ = sess.run(one)
    featInfo = tf.train.Example.FromString(one_).features.feature # not dict
    for key in featInfo:
        info_list=[(key, attr, featInfo[key].__getattribute__(attr).value) for attr in ["bytes_list","float_list","int64_list"]]
        info = [i for i in info_list if i[-1] != []][0] # 只应该有一个
        res.append("[featName]:{} [type]:{} [value_example]:{}".format(*info))

print("\n".join(sorted(res)))
    

[featName]:dnnCrsCurRoad [type]:int64_list [value_example]:[8593260L]
[featName]:dnnCrsDragRoad [type]:int64_list [value_example]:[-1L]
[featName]:dnnCrsPoiRoad [type]:int64_list [value_example]:[-1L]
[featName]:dnnDGlobalTime [type]:int64_list [value_example]:[42553L]
[featName]:dnnNpoiDis [type]:float_list [value_example]:[0.5899999737739563]
[featName]:dnnPGlobalTime [type]:int64_list [value_example]:[8655390L]
[featName]:dnncCurrentDis [type]:float_list [value_example]:[14.9399995803833]
[featName]:dnncPoiDis [type]:float_list [value_example]:[12.050000190734863]
[featName]:dnncRgeoDis [type]:float_list [value_example]:[0.0]
[featName]:dnnpidDropoffHeat [type]:float_list [value_example]:[0.0]
[featName]:dnnpidDropoffRatio [type]:float_list [value_example]:[0.0]
[featName]:dnnpidLBChargeTime [type]:float_list [value_example]:[0.0]
[featName]:dnnpidLFChargeTime [type]:float_list [value_example]:[0.0]
[featName]:dnnpidPickupHeat [type]:float_list [value_example]:[0.0]
[featName]:dnnpi

## 解析

In [19]:
trd_dir = "/nfs/map-tmp-vol0/weizhipeng/tensorflow_lr/trainDatas/BR_pidtwo_train"
trd_fp = [os.path.join(trd_dir,i) for i in os.listdir(trd_dir)]
wide_size = 35
config_path = open('/nfs/map-tmp-vol0/weizhipeng/tensorflow_lr/src/main/resources/features.json','r')
# config_path = open('/tmp-data/zhoutongzt/DD/tensorflow_lr/src/main/resources/features_oriFeat.json','r')
conf = json.load(config_path)
features={}
features['wideFeatures'] = tf.io.FixedLenFeature([wide_size], tf.int64)
# features['label'] = tf.io.FixedLenFeature([1], tf.int64)
for deepName in conf["deepFeat"]:
    fshape = conf["varlensFeat_length"].get(deepName, None)
    fshape = [] if fshape is None else [fshape]
    fdtype = tf.float32 if deepName in conf["floatFeat"] else tf.int64
    features[deepName] = tf.io.FixedLenFeature(fshape, fdtype)

print("[features.keys]: "+", ".join(sorted(features.keys())))
def _parse_func(inp):
    parsed_features = tf.io.parse_single_example(inp,features)
    return parsed_features

trd=tf.data.TFRecordDataset(trd_fp,compression_type='GZIP').map(_parse_func)

####################
# tf1.x specific
####################
dataset = trd.batch(64,drop_remainder=True)
line = dataset.make_one_shot_iterator().get_next()
to_print = ["dnnCrsCurRoad",]
with tf.Session() as sess:
    i = sess.run(line)
    print(">>> all keys: {}\n".format(",".join(i.keys())))
    for key in to_print:
        print(">>> {}:".format(key))
        print(i[key])


[features.keys]: dnnCrsCurRoad, dnnCrsDragRoad, dnnCrsPoiRoad, dnnDGlobalTime, dnnNpoiDis, dnnPGlobalTime, dnncCurrentDis, dnncPoiDis, dnncRgeoDis, dnnpidDropoffHeat, dnnpidDropoffRatio, dnnpidLBChargeTime, dnnpidLFChargeTime, dnnpidPickupHeat, dnnpidRatio, dnnpidTrajNum, dnnpidTrajRatio, dnnpoiDropoffRatio, dnnpoiRatio, dnnpsw, link, locHash, pickupHash, pid, poi, rgeoHash, wideFeatures, wifi
>>> all keys: rgeoHash,pid,dnnNpoiDis,dnncRgeoDis,poi,dnncPoiDis,dnnpidDropoffRatio,wideFeatures,dnncCurrentDis,dnnpidDropoffHeat,locHash,dnnPGlobalTime,dnnDGlobalTime,dnnpoiDropoffRatio,pickupHash,dnnpidLFChargeTime,dnnpidPickupHeat,link,dnnpidTrajRatio,dnnCrsDragRoad,dnnCrsPoiRoad,dnnpidTrajNum,dnnpidRatio,wifi,dnnpsw,dnnCrsCurRoad,dnnpidLBChargeTime,dnnpoiRatio

>>> dnnCrsCurRoad:
[9414388 9414388 9414388 9414388 9414389 9414388 9414388 9414388 9414388
 9414388 9414388 9414389 9414388 9414388 9414389 9414388 9414389 9414388
 9414389 9414388 9414388 9414389 9414389 9414389 9414389 9414388 94143

# 处理TFRecord | tf2.x

## 检查schema

In [ ]:
trd_dir = "/nfs/map-tmp-vol0/zhoutong/Data/BR_useReqTime_addCross_addTimeByFreq_train"
trd_fp = [os.path.join(trd_dir,i) for i in os.listdir(trd_dir)]
trd_unparsed=tf.data.TFRecordDataset(trd_fp,compression_type='GZIP')

for raw_record in trd_unparsed.take(1):
    example = tf.train.Example()
    _ = example.ParseFromString(raw_record.numpy())
    
def format_exampleInfo(exampleInfo):
    res = []
    for name,feat in exampleInfo.features.feature.items():
        info_list=[(name, attr, feat.__getattribute__(attr).value) for attr in ["bytes_list","float_list","int64_list"]]
        info = [(name,attr,value) for name,attr,value in info_list if value != []][0]
        res.append("[featName]:{} [type]:{} [value_example]:{}".format(*info))
    print("\n".join(sorted(map(str,res))))
    
format_exampleInfo(example)  

## 解析

In [ ]:
trd_dir = "/nfs/map-tmp-vol0/zhoutong/Data/BR_useReqTime_addCross_addTimeByFreq_addTimeHeatCross_test"
trd_fp = [os.path.join(trd_dir,i) for i in os.listdir(trd_dir)]
wide_size = 48
config_path = open('/tmp-data/zhoutongzt/DD/tensorflow_lr/src/main/resources/features_useReqTime_addCross.json','r')
# config_path = open('/tmp-data/zhoutongzt/DD/tensorflow_lr/src/main/resources/features_oriFeat.json','r')
conf = json.load(config_path)
features={}
features['wideFeatures'] = tf.io.FixedLenFeature([wide_size], tf.int64)
features['label'] = tf.io.FixedLenFeature([1], tf.int64)
for deepName in conf["deepFeat"]:
    fshape = conf["varlensFeat_length"].get(deepName, None)
    fshape = [] if fshape is None else [fshape]
    fdtype = tf.float32 if deepName in conf["floatFeat"] else tf.int64
    features[deepName] = tf.io.FixedLenFeature(fshape, fdtype)

print("[features.keys]: "+", ".join(sorted(features.keys())))
def _parse_func(inp):
    parsed_features = tf.io.parse_single_example(inp,features)
    return parsed_features

trd=tf.data.TFRecordDataset(trd_fp,compression_type='GZIP').map(_parse_func)

####################
# tf2.x specific
####################
for i in trd.take(1):
    print(i)
    print("\n[keys]:" + str(i.keys()))
    print("\n[heat_time_idx_poi_id]: " + str(i['heat_time_idx_poi_id']))
    print("[heat_time_idx_pickup_geoHash]: " + str(i['heat_time_idx_pickup_geoHash']))
    print("[heat_time_idx_bind_line_id_single]: " + str(i['heat_time_idx_bind_line_id_single']))
#     print("[weights]:" + str(i['weights']))
#     print("[order_info]:" + str(i['order_info']))
    print(i['wideFeatures'].shape)
    
# 跑不出来377w数太多了？
# set([i['wideFeatures'].shape for i in trd.as_numpy_iterator()])

# feature_column TEST

## categorical_column_with_hash_bucket

In [19]:
num_buckets_deep=7
dims_embs_deep=4
_batch_size=2

# features
features = {}
features["featA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featC"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featD"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featE"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["pickupHash"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features

{'featA': array([[2],
        [6]]), 'featB': array([[4],
        [2]]), 'featC': array([[5],
        [6]]), 'featD': array([[0],
        [4]]), 'featE': array([[4],
        [6]]), 'pickupHash': array([[3],
        [5]])}

In [26]:
tf.reset_default_graph()

# codec column

featA_cat_id_column = fc.categorical_column_with_identity("featA", num_buckets_deep, 0)
featA_cat_hash_column = fc.categorical_column_with_hash_bucket("featA", hash_bucket_size=10, dtype=tf.int64)

with tf.Session():
    fc.input_layer(features, fc.indicator_column(featA_cat_id_column))
    fc.input_layer(features, fc.indicator_column(featA_cat_hash_column))
    fc.input_layer(features, fc.indicator_column(featA_cat_id_column)).eval()
    fc.input_layer(features, fc.indicator_column(featA_cat_hash_column)).eval()

<tf.Tensor 'input_layer/concat:0' shape=(2, 7) dtype=float32>

<tf.Tensor 'input_layer_1/concat:0' shape=(2, 10) dtype=float32>

array([[0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

## cross_column

### basic

In [5]:
num_buckets_deep=7
dims_embs_deep=4
_batch_size=2

# features
features = {}
features["featA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featC"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featD"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featE"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features

{'featA': array([[0],
        [4]]), 'featB': array([[0],
        [5]]), 'featC': array([[0],
        [5]]), 'featD': array([[4],
        [2]]), 'featE': array([[4],
        [5]])}

### Run

In [7]:
tf.reset_default_graph()

# codec column
featA_cat_column = fc.categorical_column_with_identity("featA", num_buckets_deep, 0)
featA_idx_column = fc.indicator_column(featA_cat_column)

featB_cat_column = fc.categorical_column_with_identity("featB", num_buckets_deep, 0)
featB_idx_column = fc.indicator_column(featB_cat_column)

featC_cat_column = fc.categorical_column_with_identity("featC", num_buckets_deep, 0)
featC_idx_column = fc.indicator_column(featC_cat_column)

featAxB_cat_column = fc.crossed_column(["featA","featB"], hash_bucket_size=10)
featAxB_idx_column = fc.indicator_column(featAxB_cat_column)
featAxB_emb_column = fc.embedding_column(featAxB_cat_column, dimension=dims_embs_deep)

featAxBxC_cat_column = fc.crossed_column(["featA","featB","featC"], hash_bucket_size=10)
featAxBxC_idx_column = fc.indicator_column(featAxBxC_cat_column)
featAxBxC_emb_column = fc.embedding_column(featAxBxC_cat_column, dimension=dims_embs_deep)

shaerd_emb_columns = fc.shared_embedding_columns([featAxB_cat_column, featAxBxC_cat_column], dimension=dims_embs_deep)

to_be_input = [featA_idx_column,
               featB_idx_column,
               featC_idx_column,
               featAxB_idx_column,
               featAxB_emb_column,
               featAxBxC_idx_column,
               featAxBxC_emb_column]+shaerd_emb_columns
inp_layer_list = [fc.input_layer(features,i) for i in to_be_input]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    inp_list = sess.run(inp_layer_list)
    for k,v in zip([i.name for i in to_be_input], inp_list):
        print(">>> {}:\n{}".format(k,v))

>>> featA_indicator:
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]
>>> featB_indicator:
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
>>> featC_indicator:
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
>>> featA_X_featB_indicator:
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
>>> featA_X_featB_embedding:
[[-0.263117   -0.3976633   0.331601   -0.29276696]
 [-0.1234642  -0.6340207   0.10317452  0.33643612]]
>>> featA_X_featB_X_featC_indicator:
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
>>> featA_X_featB_X_featC_embedding:
[[-0.27891752  0.22432825  0.6575673  -0.6044132 ]
 [-0.27891752  0.22432825  0.6575673  -0.6044132 ]]
>>> featA_X_featB_shared_embedding:
[[-0.2952471   0.674401   -0.17939782  0.19393241]
 [ 0.00449466 -0.19109586 -0.43362102 -0.10811146]]
>>> featA_X_featB_X_featC_shared_embedding:
[[ 0.33795786  0.8862156  -0.27822748 -0.13059047]
 [ 0.33795786  0.8862156  -0.27822748 -0.13059047]]


## Interact emb by Add | fake Positional-Emb

### basic

In [4]:
##############
# h-params
##############
num_buckets_deep=20
dims_embs_deep=4
_batch_size=2

##############
# features
##############
features = {}
features["featA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["link"] = [[-1],[-1]]
features["pickupHash"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["hour_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["time_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["whateverA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["whateverB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["numericA"] = [[0.5],[1.2]]
features["numericB"] = [[0.5],[0.2]]
print(features)

################
# build columns
################
tf.reset_default_graph()
# numeric column
fc_numeric = list()
fc_numeric.append(fc.numeric_column("numericA", default_value=0))
fc_numeric.append(fc.numeric_column("numericB", default_value=0))

# codec column
emb_feat_list = list()
for i in ["featA","featB","link","pickupHash","hour_idx","time_idx"]:
    emb_feat_list.append(fc.categorical_column_with_identity(i, num_buckets_deep, default_value=0))

# emb column
fc_shared_emb = fc.shared_embedding_columns(
                    emb_feat_list,
                    dimension=dims_embs_deep, max_norm=np.sqrt(dims_embs_deep),
                    trainable=True)

# shared_emb 转成KV结构方便后面操作
fc_emb_dict = {seCol.categorical_column.key: seCol for seCol in fc_shared_emb}

# 指出将要相加的emb
emb_to_add=[['link','hour_idx'],
            ['pickupHash','time_idx']]

{'featA': array([[19],
       [19]]), 'pickupHash': array([[15],
       [ 8]]), 'time_idx': array([[9],
       [1]]), 'numericB': [[0.5], [0.2]], 'numericA': [[0.5], [1.2]], 'link': [[-1], [-1]], 'hour_idx': array([[5],
       [2]]), 'whateverB': array([[15],
       [ 0]]), 'whateverA': array([[17],
       [16]]), 'featB': array([[1],
       [2]])}


### input_layer搞事情

In [6]:
emb_added = []
for f1,f2 in emb_to_add:
    net1 = fc.input_layer(features, fc_emb_dict[f1])
    net2 = fc.input_layer(features, fc_emb_dict[f2])
    emb_added.append(net1+net2)

emb_to_add_flatten = [i for sub in emb_to_add for i in sub]
net_ori = fc.input_layer(features, [v for k,v in fc_emb_dict.items() if k not in emb_to_add_flatten])

net = tf.concat([net_ori]+emb_added,axis=1)

### run起来看看结果

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    "net_ori",sess.run(net_ori)
    "emb_added",sess.run(emb_added)
    "net",sess.run(net)
    print(">>> details of emb_added")
    for f1,f2 in emb_to_add:
        net1 = fc.input_layer(features, fc_emb_dict[f1])
        net2 = fc.input_layer(features, fc_emb_dict[f2])
        f1,sess.run(net1)
        f2,sess.run(net2)

('net_ori',
 array([[-0.03376518, -0.75637287,  0.01980731,  0.57804877, -0.41253054,
          0.76133245,  0.21623057,  0.34377784],
        [-0.03376518, -0.75637287,  0.01980731,  0.57804877,  0.5149282 ,
         -0.06459129, -0.2458921 , -0.7198298 ]], dtype=float32))

('emb_added', [array([[-0.2679743 , -0.51504064,  0.3350453 ,  0.47310346],
         [ 0.5149282 , -0.06459129, -0.2458921 , -0.7198298 ]],
        dtype=float32),
  array([[-0.02274224,  1.3028573 , -0.6851692 ,  0.33623815],
         [ 0.28992552,  1.1738838 ,  0.27457008,  0.5552162 ]],
        dtype=float32)])

('net',
 array([[-0.03376518, -0.75637287,  0.01980731,  0.57804877, -0.41253054,
          0.76133245,  0.21623057,  0.34377784, -0.2679743 , -0.51504064,
          0.3350453 ,  0.47310346, -0.02274224,  1.3028573 , -0.6851692 ,
          0.33623815],
        [-0.03376518, -0.75637287,  0.01980731,  0.57804877,  0.5149282 ,
         -0.06459129, -0.2458921 , -0.7198298 ,  0.5149282 , -0.06459129,
         -0.2458921 , -0.7198298 ,  0.28992552,  1.1738838 ,  0.27457008,
          0.5552162 ]], dtype=float32))

>>> details of emb_added


('link', array([[0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32))

('hour_idx', array([[-0.2679743 , -0.51504064,  0.3350453 ,  0.47310346],
        [ 0.5149282 , -0.06459129, -0.2458921 , -0.7198298 ]],
       dtype=float32))

('pickupHash', array([[-0.5041411 ,  0.9702361 , -0.7937973 , -0.33923268],
        [ 0.70245606,  0.41255134,  0.05833951,  0.21143836]],
       dtype=float32))

('time_idx', array([[ 0.48139885,  0.33262116,  0.10862812,  0.6754708 ],
        [-0.41253054,  0.76133245,  0.21623057,  0.34377784]],
       dtype=float32))

## Interact emb by Inner-product

### basic

In [4]:
num_buckets_deep=7
dims_embs_deep=3
_batch_size=2

# features
features = {}
features["featA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featC"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["link"] = [[-1],[2]]
assert(len(features["link"]) == _batch_size)
features["pickupHash"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["hour_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["time_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))

for k,v in features.items():
    print(k)
    print(v)

featA
[[1]
 [6]]
pickupHash
[[6]
 [2]]
featB
[[5]
 [5]]
link
[[-1], [2]]
featC
[[3]
 [2]]
hour_idx
[[0]
 [5]]
time_idx
[[4]
 [5]]


In [5]:
tf.reset_default_graph()
# numeric column
fc_numeric = list()
fc_numeric.append(fc.numeric_column("numericA", default_value=0))
fc_numeric.append(fc.numeric_column("numericB", default_value=0))

# codec column
emb_feat_list = list()
for i in ["featA","featB","featC","link","pickupHash","hour_idx","time_idx"]:
    emb_feat_list.append(fc.categorical_column_with_identity(i, num_buckets_deep, default_value=0))

# emb column
fc_shared_emb = fc.shared_embedding_columns(
                    emb_feat_list,
                    dimension=dims_embs_deep, max_norm=np.sqrt(dims_embs_deep),
                    trainable=True)

# shared_emb 转成KV结构方便后面操作
fc_emb_dict = {seCol.categorical_column.key: seCol for seCol in fc_shared_emb}

# 指出将要相加的emb
emb_to_add=[['link','hour_idx'],
            ['pickupHash','time_idx']]


### input_layer操作起来

format by: [["hour_idx","featA"], ["hour_idx","featB"], ["hour_idx", "featC"]]

In [131]:
tf.reset_default_graph()
emb_to_innerProduct = ["link","pickupHash"]
emb_to_innerProduct = [[i,"hour_idx"] for i in emb_to_innerProduct]

with tf.Session() as sess:
    emb_dot = []
    for f1,f2 in emb_to_innerProduct:
        net1 = fc.input_layer(features, fc_emb_dict[f1])
        net2 = fc.input_layer(features, fc_emb_dict[f2])
        _res = net1 * net2
        res = tf.reduce_sum(net1*net2, axis=1)
        emb_dot.append(res)

    sess.run(tf.global_variables_initializer())
    print(">>> last-one as demo")
    print(f1)
    print(sess.run(net1))
    print(f2)
    print(sess.run(net2))
    print("_res | f1 f2做内积得到")
    print(sess.run(_res))
    print("res | _res做reduce_sum得到")
    print(sess.run(res))
    
    
    print(">>> emb_dot")
    print(sess.run(emb_dot))
    fm_2nd_res = tf.reduce_sum(emb_dot, axis=0)
    print(">>> fm_2nd_res")
    print(sess.run(fm_2nd_res))

>>> last-one as demo
pickupHash
[[-0.06088226 -0.696076   -0.45172015]
 [-0.915245   -0.16775647  0.3668979 ]]
hour_idx
[[-0.5740352   0.24671505  0.4600736 ]
 [ 0.27878988 -0.06976111 -0.5711166 ]]
_res | f1 f2做内积得到
[[ 0.03494856 -0.17173243 -0.20782451]
 [-0.25516105  0.01170288 -0.2095415 ]]
res | _res做reduce_sum得到
[-0.34460837 -0.45299965]
>>> emb_dot
[array([0.        , 0.28957057], dtype=float32), array([-0.34460837, -0.45299965], dtype=float32)]
>>> fm_2nd_res
[-0.34460837 -0.16342908]


### input_layer操作起来 | 字典形式

format by: {"hour_idx": ["featA", "featB", "featC"], "time_idx": ["featA", "featB", "featC"]}

In [19]:
help(tf.contrib.layers.batch_norm)

Help on function batch_norm in module tensorflow.contrib.layers.python.layers.layers:

batch_norm(*args, **kwargs)
    Adds a Batch Normalization layer from http://arxiv.org/abs/1502.03167.
    
      "Batch Normalization: Accelerating Deep Network Training by Reducing
      Internal Covariate Shift"
    
      Sergey Ioffe, Christian Szegedy
    
    Can be used as a normalizer function for conv2d and fully_connected. The
    normalization is over all but the last dimension if `data_format` is `NHWC`
    and all but the second dimension if `data_format` is `NCHW`.  In case of a 2D
    tensor this corresponds to the batch dimension, while in case of a 4D tensor
    this
    corresponds to the batch and space dimensions.
    
    Note: when training, the moving_mean and moving_variance need to be updated.
    By default the update ops are placed in `tf.GraphKeys.UPDATE_OPS`, so they
    need to be added as a dependency to the `train_op`. For example:
    
    ```python
      update_ops 

In [26]:
tf.reset_default_graph()
from tensorflow.python.keras import regularizers
from tensorflow.python.ops import init_ops
emb_to_innerProduct = {"hour_idx":["featA","featB","featC","link","pickupHash"],
                       "time_idx":["featA","featB","featC","link","pickupHash"],}

with tf.Session() as sess:
    emb_dot = []
    for k, v in emb_to_innerProduct.items():
        # k: f1  v: [featA,featB,featC]
        # cross: f1*featA + f1*featB + f1*featC 
        emb1 = fc.input_layer(features, fc_emb_dict[k])
        emb2 = fc.input_layer(features, [fc_emb_dict[i] for i in v])
        emb1_r = tf.expand_dims(emb1, -1)
        emb2_r = tf.reshape(emb2,[-1, len(v), emb1.shape[-1]])
        res = tf.squeeze(tf.matmul(emb2_r, emb1_r))
        emb_dot.append(res)
    all_emb_dot = tf.concat(emb_dot, axis=1)
#     fm_2nd_res = tf.reduce_sum(tf.concat(emb_dot, axis=1), axis=1)
#     fm_2nd_res = tf.reshape(fm_2nd_res, [-1,1])
    
    logits = tf.layers.dense(all_emb_dot, 4, kernel_initializer=init_ops.glorot_uniform_initializer(),
                        activation=None, kernel_regularizer=regularizers.l2(0.01),
                        name="logits")

    all_emb_dot_bn = tf.contrib.layers.batch_norm(all_emb_dot,is_training=False,updates_collections=None)
    logits_bn = tf.layers.dense(all_emb_dot_bn, 4, kernel_initializer=init_ops.glorot_uniform_initializer(),
                        activation=None, kernel_regularizer=regularizers.l2(0.01),
                        name="logits_bn")
    

In [30]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.matmul(emb2_r, emb1_r)
    res
    sess.run(tf.matmul(emb2_r, emb1_r))
    sess.run(tf.reshape(tf.matmul(emb2_r, emb1_r),[-1, tf.matmul(emb2_r, emb1_r).shape[1]]))

<tf.Tensor 'MatMul_5:0' shape=(2, 5, 1) dtype=float32>

<tf.Tensor 'Squeeze_1:0' shape=(2, 5) dtype=float32>

array([[[-0.4614984 ],
        [-0.13459314],
        [ 0.8183997 ],
        [ 0.        ],
        [-0.1535201 ]],

       [[-0.03328903],
        [ 0.82868665],
        [ 0.55066764],
        [ 0.55066764],
        [ 0.55066764]]], dtype=float32)

array([[-0.4614984 , -0.13459314,  0.8183997 ,  0.        , -0.1535201 ],
       [-0.03328903,  0.82868665,  0.55066764,  0.55066764,  0.55066764]],
      dtype=float32)

In [25]:
print(k)
print(v)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("emb1")
    sess.run(emb1)
    print("emb2")
    sess.run(emb2)
    print("emb1_r")
    sess.run(emb1_r)
    print("emb2_r")
    sess.run(emb2_r)
    print("res")
    sess.run(res)
    
    print(">>> 只看batch里的第一个")
    print("emb1")
    sess.run(emb1)[0]
#     print("emb2")
#     sess.run(emb2)[0]
#     print("emb1_r")
#     sess.run(emb1_r)[0]
    print("emb2_r")
    sess.run(emb2_r)[0]
    print("res")
    sess.run(res)[0]
    
    print("emb_dot")
    sess.run(emb_dot)
    print("all_emb_dot | concat emb_dot")
    sess.run(all_emb_dot)
#     print("tf.concat(emb_dot, axis=0)")
#     sess.run(tf.concat(emb_dot, axis=1))
#     print("fm_2nd_res | tf.reduce_sum(tf.concat(emb_dot, axis=1), axis=1)")
#     sess.run(fm_2nd_res)
    print("logits | tf.concat(all_emb_dot, axis=1) & 4分类")
    sess.run(logits)

    print("all_emb_dot_bn | concat emb_dot")
    sess.run(all_emb_dot_bn)
    print("logits_bn | tf.concat(all_emb_dot_bn, axis=1) & 4分类")
    sess.run(logits_bn)

time_idx
['featA', 'featB', 'featC', 'link', 'pickupHash']
emb1


array([[ 0.41871086,  0.52999526,  0.4945093 ],
       [-0.629783  , -0.06730537, -0.3083893 ]], dtype=float32)

emb2


array([[ 0.00486276,  0.46456635,  0.63068444, -0.629783  , -0.06730537,
        -0.3083893 ,  0.79900736, -0.19154112, -0.7691954 ,  0.        ,
         0.        ,  0.        ,  0.26811442, -0.47049198, -0.89255047],
       [ 0.26811442, -0.47049198, -0.89255047, -0.629783  , -0.06730537,
        -0.3083893 , -0.27455962,  0.29733664, -0.7572018 , -0.27455962,
         0.29733664, -0.7572018 , -0.27455962,  0.29733664, -0.7572018 ]],
      dtype=float32)

emb1_r


...

array([[-0.03423132,  0.16598332,  0.17956713, -0.2545311 ],
       [ 0.16397662, -0.02424657, -1.1411203 ,  0.46004915]],
      dtype=float32)

all_emb_dot_bn | concat emb_dot


array([[ 0.12440307, -0.4895905 ,  0.30324197,  0.        , -0.03026951,
         0.55985355, -0.45164412, -0.1472635 ,  0.        , -0.5781816 ],
       [ 0.13799678,  0.49601263,  0.38622048,  0.38622048,  0.38622048,
         0.13799678,  0.49601263,  0.38622048,  0.38622048,  0.38622048]],
      dtype=float32)

logits_bn | tf.concat(all_emb_dot_bn, axis=1) & 4分类


array([[-0.04481216,  0.5127205 , -0.04209074, -0.35768437],
       [-0.33998445, -0.2264207 , -0.0203252 , -0.3708216 ]],
      dtype=float32)

In [21]:
sum([ 0.12115489,  0.39662713, -0.45657066,  0.        , -0.11924169,
        -0.06230509, -0.43095323,  0.20267147,  0.        ,  0.1147818 ])
sum([ 0.98669964, -0.9935233 , -0.96973526,  0.9557099 ,  0.80604684,
         0.95365196, -0.9989523 ,  0.9882004 ,  0.9557099 ,  0.98104113])

-0.2338353800000001

3.66484891

In [135]:
# copy 出来手工计算
emb1 = [ 0.91038424, -0.03854756, -0.2341526 ]
emb2 = [ 0.627122  ,  0.35224104,  0.40040714]

[a*b for a,b in zip(emb1,emb2)]
sum([a*b for a,b in zip(emb1,emb2)])

# sum([ 0.3972234 , -0.308128  ,  0.        , -0.07037204, -0.308128  ,
#          1.4594309 ,  0.        , -0.29277474])
# sum([ 0.5564581 ,  0.41053557,  0.        ,  0.5564581 ,  0.4649981 , 0.17164305,  0.        ,  0.4649981 ])

[0.5709219853572799, -0.013578032623862401, -0.093756372889564]

0.4635875798438535

### Run

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    "net",sess.run(net)
    "emb_added",sess.run(emb_added)
    "net_final",sess.run(net_final)
    print(">>> details of emb_added")
    for f1,f2 in emb_to_add:
        net1 = fc.input_layer(features, fc_emb_dict[f1])
        net2 = fc.input_layer(features, fc_emb_dict[f2])
        f1,sess.run(net1)
        f2,sess.run(net2)

In [ ]:
# def fm(inputs):
#     if K.ndim(inputs) != 3:
#         raise ValueError(
#             "Unexpected inputs dimensions %d, expect to be 3 dimensions"
#             % (K.ndim(inputs)))

#     concated_embeds_value = inputs

#     square_of_sum = tf.square(reduce_sum(
#         concated_embeds_value, axis=1, keep_dims=True))
#     sum_of_square = reduce_sum(
#         concated_embeds_value * concated_embeds_value, axis=1, keep_dims=True)
#     cross_term = square_of_sum - sum_of_square
#     cross_term = 0.5 * reduce_sum(cross_term, axis=2, keep_dims=False)

#     return cross_term



## Interact emb by Co-Action Unit

### basic | imitation hparams

In [5]:
hparams = {}
hparams['batch_size'] = 2
hparams['n_classes'] = [4,4]  # idx=1是跨路label
hparams['embedding_size'] = 4
hparams["embedding_feature_size"]=20
hparams['emb_mlp_units'] = [8,8,8]
hparams['emb_mlp_feature_size'] = 100

hparams

{'batch_size': 2,
 'emb_mlp_feature_size': 100,
 'emb_mlp_units': [8, 8, 8],
 'embedding_feature_size': 20,
 'embedding_size': 4,
 'n_classes': [4, 4]}

### basic

In [6]:
##############
# h-params
##############
num_buckets_deep=hparams["embedding_feature_size"]
dims_embs_deep=hparams['embedding_size']
_batch_size=hparams['batch_size']

##############
# features
##############
features = {}
features["featA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["featB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["link"] = [[-1],[-1]]
features["pickupHash"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["hour_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["time_idx"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["whateverA"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["whateverB"] = np.random.randint(num_buckets_deep, size=(_batch_size, 1))
features["numericA"] = [[0.5],[1.2]]
features["numericB"] = [[0.5],[0.2]]
print(features)

################
# build columns
################
tf.reset_default_graph()
# numeric column
fc_numeric = list()
fc_numeric.append(fc.numeric_column("numericA", default_value=0))
fc_numeric.append(fc.numeric_column("numericB", default_value=0))

# codec column
emb_feat_list = list()
for i in ["featA","featB","link","pickupHash","hour_idx","time_idx"]:
    emb_feat_list.append(fc.categorical_column_with_identity(i, num_buckets_deep, default_value=0))

# emb column
fc_shared_emb = fc.shared_embedding_columns(
                    emb_feat_list,
                    dimension=dims_embs_deep, max_norm=np.sqrt(dims_embs_deep),
                    trainable=True)

# shared_emb 转成KV结构方便后面操作
fc_emb_dict = {seCol.categorical_column.key: seCol for seCol in fc_shared_emb}

# 指出将要操作的emb
emb_to_add=[['link','hour_idx'],
            ['pickupHash','time_idx']]

{'featA': array([[1],
       [6]]), 'pickupHash': array([[18],
       [ 5]]), 'time_idx': array([[16],
       [17]]), 'numericB': [[0.5], [0.2]], 'numericA': [[0.5], [1.2]], 'link': [[-1], [-1]], 'hour_idx': array([[15],
       [ 3]]), 'whateverB': array([[14],
       [14]]), 'whateverA': array([[5],
       [6]]), 'featB': array([[ 5],
       [17]])}


In [7]:
def co_action_unit_fc(hparams):
    # fc_target_item = fc.categorical_column_with_identity(hparams["emb_mlp_feat"], hparams['mlp_embedding_feature_size'], default_value=0)
    fc_target_item = fc.categorical_column_with_hash_bucket("pickupHash", hash_bucket_size=hparams['emb_mlp_feature_size'], dtype=tf.int64)

    fc_emb_mlp_kernel_list = []
    fc_emb_mlp_bias_list = []
    inp_size = hparams['embedding_size']
    for idx, i in enumerate(hparams['emb_mlp_units']):
        kernel_size = inp_size*i
        bias_size = i
        _fc_emb_kernel = fc.embedding_column(fc_target_item,
                                             dimension=kernel_size, max_norm=np.sqrt(kernel_size),
                                             trainable=True)
        fc_emb_mlp_kernel_list.append(_fc_emb_kernel)
        _fc_emb_bias = fc.embedding_column(fc_target_item,
                                           dimension=bias_size, max_norm=np.sqrt(bias_size),
                                           trainable=True)
        fc_emb_mlp_bias_list.append(_fc_emb_bias)

        # # >>>deubg
        # kernel = np.random.random([inp_size,i])
        # bias = np.random.random(bias_size)
        # print("kernel shape:%s size:%s" % (kernel.shape, kernel_size))
        # print("bias shape:%s size:%s" % (bias.shape, bias_size))
        # # <<<deubg

        inp_size = i  # IMPORTANT

    # [fc_emb.dimension for fc_emb in fc_emb_mlp_kernel_list]
    # [fc_emb.dimension for fc_emb in fc_emb_mlp_bias_list]
    return fc_emb_mlp_kernel_list, fc_emb_mlp_bias_list
fc_emb_mlp_kernel_list, fc_emb_mlp_bias_list = co_action_unit_fc(hparams)
fc_emb_mlp_kernel_list
fc_emb_mlp_bias_list

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=32, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64b18>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=5.656854249492381, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=64, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64c08>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=8.0, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=64, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64938>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=8.0, trainable=True)]

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=8, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64ed8>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=2.8284271247461903, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=8, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64e60>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=2.8284271247461903, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='pickupHash', hash_bucket_size=100, dtype=tf.int64), dimension=8, combiner='mean', layer_creator=<function _creator at 0x7f94cdf64d70>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=2.8284271247461903, trainable=True)]

In [12]:
fc_emb_inp_list = [fc_emb_dict[i] for i in ['featA','featB']]

emb_inp = fc.input_layer(features, fc_emb_inp)  # (batch_size, D)
emb_inp = tf.math.pow(emb_inp, 1) + tf.math.pow(emb_inp, 2) + tf.math.pow(emb_inp, 3)
emb_inp = tf.expand_dims(emb_inp, 1)

net = emb_inp
for idx, fc_emb in enumerate(fc_emb_mlp_kernel_list):
    emb_mlp_kernel = fc.input_layer(features, fc_emb)
    cur_output = hparams['emb_mlp_units'][idx]
    emb_mlp_kernel = tf.reshape(emb_mlp_kernel, [-1, fc_emb.dimension/cur_output, cur_output])
    emb_mlp_kernel
#     emb_mlp_bias = fc.input_layer(features, fc_emb_mlp_bias_list[idx])
#     emb_mlp_bias = tf.expand_dims(emb_mlp_bias, 1)
#     # wx+b
#     res = tf.einsum("bik,bkj->bij", net, emb_mlp_kernel) + emb_mlp_bias
#     # activation
#     res = tf.nn.relu(res)
#     net = res
# net = tf.squeeze(net)
net

<tf.Tensor 'Reshape_2:0' shape=(2, 4, 8) dtype=float32>

<tf.Tensor 'Reshape_3:0' shape=(2, 8, 8) dtype=float32>

<tf.Tensor 'Reshape_4:0' shape=(2, 8, 8) dtype=float32>

<tf.Tensor 'ExpandDims_5:0' shape=(2, 1, 8) dtype=float32>

In [14]:
def co_action_unit(features, fc_emb_inp, fc_emb_mlp_kernel_list, fc_emb_mlp_bias_list, hparams, mode):
    """

    :param features:
    :param fc_emb_inp: emb_feature_column
    :param fc_emb_mlp_kernel_list: list[emb_feature_column]
    :param fc_emb_mlp_bias_list: list[emb_feature_column]
    :param hparams: params
    :param mode: TRAIN EVALUATION .etc
    :return:
    """
    emb_inp = fc.input_layer(features, fc_emb_inp)  # (batch_size, D)
    emb_inp = tf.math.pow(emb_inp, 1) + tf.math.pow(emb_inp, 2) + tf.math.pow(emb_inp, 3)
    emb_inp = tf.expand_dims(emb_inp, 1)

    net = emb_inp
    for idx, fc_emb in enumerate(fc_emb_mlp_kernel_list):
        emb_mlp_kernel = fc.input_layer(features, fc_emb)
        cur_output = hparams['emb_mlp_units'][idx]
        emb_mlp_kernel = tf.reshape(emb_mlp_kernel, [-1, fc_emb.dimension/cur_output, cur_output])
        emb_mlp_bias = fc.input_layer(features, fc_emb_mlp_bias_list[idx])
        emb_mlp_bias = tf.expand_dims(emb_mlp_bias, 1)
        # wx+b
        res = tf.einsum("bik,bkj->bij", net, emb_mlp_kernel) + emb_mlp_bias
        # activation
        res = tf.nn.relu(res)
        net = res
    net = tf.squeeze(net)
    return net

fc_emb_inp_list = [fc_emb_dict[i] for i in ['featA','featB']]
emb_can_list = []

for fc_emb_inp in fc_emb_inp_list:
        emb_can = co_action_unit(features, fc_emb_inp, fc_emb_mlp_kernel_list, fc_emb_mlp_bias_list, hparams, 'TRAIN')
        print(emb_can)
        emb_can_list.append(emb_can)

Tensor("Squeeze:0", shape=(2, 8), dtype=float32)
Tensor("Squeeze_1:0", shape=(2, 8), dtype=float32)


In [20]:
net = fc.input_layer(features, fc_emb_dict.values())
net
emb_can_list
tf.concat([net] + emb_can_list ,axis=1)

<tf.Tensor 'input_layer_28/concat:0' shape=(2, 24) dtype=float32>

[<tf.Tensor 'Squeeze:0' shape=(2, 8) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(2, 8) dtype=float32>]

<tf.Tensor 'concat:0' shape=(2, 40) dtype=float32>

### basic | emb_mlp

kernel一张表，bias一张表

In [ ]:
# emb_mlp size的计算，输入是普通的emb_size输出是n_class，中间是N个全连接
hparams['emb_mlp_units'] = [8]*3  # 先用桶状结构吧，有利于后面reshape然后用矩阵乘法计算更高效（不然要用slice）
hparams['emb_mlp_bias'] = [8]*3

emb_mlp_kernel_size_1st = 0
emb_mlp_kernel_size = 0
emb_mlp_bias_size = 0
inp_size = hparams['embedding_size']
for idx,i in enumerate(hparams['emb_mlp_units']):
    kernel_size = inp_size*i
    bias_size = i
    kernel = np.random.random([inp_size,i])
    bias = np.random.random(bias_size)
    print("shape:%s size:%s" % (kernel.shape, kernel_size))
    print("shape:%s size:%s" % (bias.shape, bias_size))
    if idx == 0:
        emb_mlp_kernel_size_1st = kernel_size
    
    total_emb_mlp_kernel_size += kernel_size
    total_emb_mlp_bias_size += bias_size
    inp_size = i

hparams['emb_mlp_kernel_size'] = total_emb_mlp_kernel_size
hparams['emb_mlp_bias_size'] = total_emb_mlp_bias_size
total_emb_mlp_kernel_size
total_emb_mlp_bias_size

每个kernel各自一张表，每个bias各自一张表

In [30]:
# emb_mlp size的计算，输入是普通的emb_size输出是n_class，中间是N个全连接
hparams['emb_mlp_units'] = [8,16,8]
# 应该是独立去重计算，但是这里不像重新建索引，就还是偷个懒用之前的size
hparams["mlp_embedding_feature_size"] = hparams["embedding_feature_size"]  

In [31]:
tf.reset_default_graph()

targetItemCol = fc.categorical_column_with_hash_bucket("pickupHash", hash_bucket_size=hparams['embedding_feature_size'], dtype=tf.int64)
fc_emb_mlp_kernel = []
fc_emb_mlp_bias = []
inp_size = hparams['embedding_size']
for idx,i in enumerate(hparams['emb_mlp_units']):
    kernel_size = inp_size*i
    bias_size = i
    _fc_emb_kernel = fc.embedding_column(targetItemCol,
                                         dimension=kernel_size, max_norm=np.sqrt(kernel_size),
                                         trainable=True)
    fc_emb_mlp_kernel.append(_fc_emb_kernel)
    _fc_emb_bias = fc.embedding_column(targetItemCol,
                                       dimension=bias_size, max_norm=np.sqrt(bias_size),
                                       trainable=True)
    fc_emb_mlp_bias.append(_fc_emb_bias)
    
    # >>>deubg
    kernel = np.random.random([inp_size,i])
    bias = np.random.random(bias_size)
    print("kernel shape:%s size:%s" % (kernel.shape, kernel_size))
    print("bias shape:%s size:%s" % (bias.shape, bias_size))
    # <<<deubg
    
    inp_size = i  # IMPORTANT

[fc_emb.dimension for fc_emb in fc_emb_mlp_kernel]
[fc_emb.dimension for fc_emb in fc_emb_mlp_bias]


# 输入侧的emb column
# fc_emb_inp = [fc_emb_dict[i] for i in ['link','hour_idx']]
fc_emb_inp = fc_emb_dict['hour_idx']  # 一个个来吧，考虑到还要做1~3次幂
emb_inp = fc.input_layer(features, fc_emb_inp) # (batch_size, D)
emb_inp = tf.math.pow(emb_inp,1) + tf.math.pow(emb_inp,2) + tf.math.pow(emb_inp,3)
emb_inp = tf.expand_dims(emb_inp,1)
emb_inp

inp = emb_inp
for idx,fc_emb in enumerate(fc_emb_mlp_kernel):
    print(">>> at idx: %s " % idx)
    emb_mlp_kernel = fc.input_layer(features, fc_emb)
    cur_output = hparams['emb_mlp_units'][idx]
    emb_mlp_kernel = tf.reshape(emb_mlp_kernel,[-1, fc_emb.dimension/cur_output, cur_output])
    emb_mlp_bias = fc.input_layer(features, fc_emb_mlp_bias[idx])
    emb_mlp_bias = tf.expand_dims(emb_mlp_bias,1)
    res = tf.einsum("bik,bkj->bij", inp, emb_mlp_kernel) + emb_mlp_bias
    res = tf.nn.relu(res)
    inp = res

res


kernel shape:(4, 8) size:32
bias shape:(8,) size:8
kernel shape:(8, 16) size:128
bias shape:(16,) size:16
kernel shape:(16, 8) size:128
bias shape:(8,) size:8


[32, 128, 128]

[8, 16, 8]

<tf.Tensor 'ExpandDims:0' shape=(2, 1, 4) dtype=float32>

>>> at idx: 0 
>>> at idx: 1 
>>> at idx: 2 


<tf.Tensor 'Relu_2:0' shape=(2, 1, 8) dtype=float32>

# Estimator

## check model

In [42]:
ckpt_dir = os.path.join(DATA_PATH,"BR_addHeat_PE_timIdxByFreqIdx_WDModel_drop_feat")


train_spec = dnn_linear_combined_model_fn(
                    features=tr_data[0],
                    labels=tr_data[1],
                    fc_wide=fc_wide,
                    fc_numeric=fc_numeric,
                    n_classes=hparams['n_classes'],
                    hparams=hparams,
                    mode=tf.estimator.ModeKeys.TRAIN,
                    fc_emb_dict=fc_emb_dict,
                    emb_to_add=emb_to_add, emb_to_innerProduct=emb_to_innerProduct)

'/tmp-data/zhoutongzt/DD/Data/BR_addHeat_PE_timIdxByFreqIdx_WDModel_drop_feat'

# API

In [56]:
[i for i in dir(tf.nn) if "entropy" in i]

['sigmoid_cross_entropy_with_logits',
 'softmax_cross_entropy_with_logits',
 'softmax_cross_entropy_with_logits_v2',
 'sparse_softmax_cross_entropy_with_logits',
 'weighted_cross_entropy_with_logits']

In [71]:
np.random.seed(1024)
logits = np.random.random([4,4])
labels=[1,1,2,3]
"logits"
logits
"labels"
labels
with tf.Session() as sess:
    "softmax(logits)"
    sess.run(tf.nn.softmax(logits))
    "target-one of softmax(logits)"
    to = [sess.run(tf.nn.softmax(logits))[idx,i] for idx,i in enumerate(labels)]
    to
    [np.log(i) for i in to]
    sess.run(tf.losses.sparse_softmax_cross_entropy(logits=logits,labels=labels))

'logits'

array([[0.64769123, 0.99691358, 0.51880326, 0.65811273],
       [0.59906347, 0.75306733, 0.13624713, 0.00411712],
       [0.14950888, 0.698439  , 0.59335256, 0.89991535],
       [0.44445739, 0.316785  , 0.92308176, 0.46586186]])

'labels'

[1, 1, 2, 3]

'softmax(logits)'

array([[0.23215253, 0.32918403, 0.20407889, 0.23458456],
       [0.29872185, 0.34845766, 0.18804786, 0.16477263],
       [0.15605622, 0.27019568, 0.24324277, 0.33050533],
       [0.22145306, 0.19491006, 0.35739264, 0.22624424]])

'target-one of softmax(logits)'

[0.32918402769283184,
 0.34845765974154597,
 0.24324277039282377,
 0.22624423686097198]

[-1.111138329907829,
 -1.05423854901307,
 -1.413695279250508,
 -1.4861401691642224]

1.2663030624389648